In [1]:
# Dependencies and General Setup
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Cleaning zipcode data
# Data files
zips_ks_path = "resources/raw data/ks_zipcodes.csv"
zips_mo_path = "resources/raw data/mo_zipcodes.csv"

# Read the data
zips_ks = pd.read_csv(zips_ks_path)
zips_mo = pd.read_csv(zips_mo_path)

# Combine the data into a single datasets
zips_df = pd.concat([zips_ks, zips_mo])
zips_df

# Exported to CSV

,Zip Code,Zip Code Type,Zip Code Name,County
0,66002,Non-Unique,Atchison,Atchison
1,66006,Non-Unique,Baldwin City,Douglas
2,66007,Non-Unique,Basehor,Leavenworth
3,66008,Non-Unique,Bendena,Doniphan
4,66010,Non-Unique,Blue Mound,Linn
...,...,...,...,...
1153,65817,PO Box,Springfield,Greene
1154,65890,Unique,Springfield,Greene
1155,65897,Unique,Springfield,Greene
1156,65898,Unique,Springfield,Greene


In [3]:
# Gathering Census Data
import requests
from census import Census
from config import census_key

c = Census(census_key, year=2013)

# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E","B17001_002E","B03003_002E","B03003_003E","B02001_002E",
                          "B02001_003E","B02001_004E","B02001_005E","B02001_006E",
                          "B02001_007E","B02001_008E","B02001_009E","B02001_010E"),
                         {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B03003_002E": "Not Hispanic or Latino",
                                      "B03003_003E": "Hispanic or Latino",
                                      "B02001_002E": "White Alone",
                                      "B02001_003E": "Black or African American Alone",
                                      "B02001_004E": "American Indian and Alaska Native Alone",
                                      "B02001_005E": "Asian Alone",
                                      "B02001_006E": "Native Hawaiian and Other Pacific Islander Alone",
                                      "B02001_007E": "Some Other Race Alone",
                                      "B02001_008E": "Two or More Races",
                                      "B02001_009E": "Two or More Races Including Some Other Race",
                                      "B02001_010E": "Two or More Races Excluding Some Other Race, and Three or More Races",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate","Not Hispanic or Latino",
                      "Hispanic or Latino","White Alone","Black or African American Alone",
                       "American Indian and Alaska Native Alone","Asian Alone",
                      "Native Hawaiian and Other Pacific Islander Alone","Some Other Race Alone",
                      "Two or More Races","Two or More Races Including Some Other Race",
                      "Two or More Races Excluding Some Other Race, and Three or More Races"]]

# Visualize
print(len(census_pd))
census_pd.head()

# Save as a csv
# Note to avoid any issues later, use encoding="utf-8"
# census_pd.to_csv("census_data.csv", encoding="utf-8", index=False)

33120


,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Not Hispanic or Latino,Hispanic or Latino,White Alone,Black or African American Alone,American Indian and Alaska Native Alone,Asian Alone,Native Hawaiian and Other Pacific Islander Alone,Some Other Race Alone,Two or More Races,Two or More Races Including Some Other Race,"Two or More Races Excluding Some Other Race, and Three or More Races"
0,08518,5217.0,41.5,74286.0,33963.0,170.0,3.258578,4928.0,289.0,3774.0,755.0,0.0,439.0,0.0,148.0,101.0,13.0,88.0
1,08520,27468.0,37.4,90293.0,37175.0,1834.0,6.676860,20887.0,6581.0,18000.0,1428.0,79.0,4117.0,17.0,2993.0,834.0,296.0,538.0
2,08525,4782.0,47.1,118656.0,59848.0,43.0,0.899205,4717.0,65.0,4481.0,142.0,0.0,108.0,0.0,1.0,50.0,3.0,47.0
3,08527,54867.0,42.2,88588.0,37021.0,2191.0,3.993293,50112.0,4755.0,48726.0,2807.0,23.0,1569.0,41.0,505.0,1196.0,334.0,862.0
4,08528,245.0,48.5,58676.0,49117.0,0.0,0.000000,245.0,0.0,245.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# Cleaning PPP Data
# https://www.kaggle.com/govtrades/sba-paycheck-protection-program-loan-data
# Cleaned PPP raw data in excel
# Import our other cleaned df's
zipcodes_df = pd.read_csv(r"resources/zip_code_data_cleaned.csv")
census_data_df = pd.read_csv(r"resources/census_data.csv")
ppp_data_df = pd.read_csv(r"resources/PPP_Data_Combined.csv")

# Combine the zipcode data and the census data to tie census info to counties
zipcodes_df = zipcodes_df.rename(columns={"Zip Code":"Zipcode"})
local_census_data_df = zipcodes_df.merge(census_data_df,on="Zipcode")

ppp_data_clean_df = ppp_data_df[["LoanRange","State","Zip","JobsReported"]].rename(columns={"Zip":"Zipcode"})

# Pull jobs reported by zip
jobs_by_zip_df = ppp_data_clean_df.groupby("Zipcode").count()

# Combine everything!
mega_df = local_census_data_df.merge(jobs_by_zip_df,on="Zipcode")

# Eliminate the unnecessary columns
manageable_df = mega_df[["Zipcode","County","Population","Poverty Count","Hispanic or Latino","White Alone","JobsReported"]]

# Merge in the state by the zip
zip_to_state_df = ppp_data_df[["Zip","State"]].drop_duplicates().rename(columns={"Zip":"Zipcode"})
manageable_df = manageable_df.merge(zip_to_state_df,on="Zipcode")

grouped_df = pd.DataFrame(manageable_df.groupby(["State","County"]).sum())

del grouped_df["Zipcode"]
grouped_df.head()

# export csv
# grouped_df.to_csv("resources/grouped_df.csv")

C:\Users\rdgg3\.conda\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Population  Poverty Count  Hispanic or Latino  White Alone  \
State County                                                                 
KS    Allen        13064.0         2141.0               395.0      12179.0   
      Anderson      8210.0         1040.0                14.0       7855.0   
      Atchison     16548.0         2567.0               449.0      15049.0   
      Barber        4407.0          488.0                96.0       4253.0   
      Barton       27808.0         3774.0              3777.0      25561.0   

                JobsReported  
State County                  
KS    Allen              195  
      Anderson           116  
      Atchison           240  
      Barber             167  
      Barton             782

In [10]:
# Gather Unemployment Data from BLS website as CSV
# https://download.bls.gov/pub/time.series/la/la.data.23.Kansas
# https://download.bls.gov/pub/time.series/la/la.data.32.Missouri

# bring in csv data
kansas_csv = "resources/Unemployment CSV and ReadMe/Kansas Unemployment Data.csv"
missouri_csv = "resources/Unemployment CSV and ReadMe/Missouri Unemployment Data.csv"

# create dataframes for KS and Mo
KS_df = pd.read_csv(kansas_csv)
MO_df = pd.read_csv(missouri_csv)

# drop unnecessary columns for readability
KS_df = KS_df.drop(columns = ['footnote_codes', 'series_id'])

# rename columns for readability
KS_df = KS_df.rename(columns = {'trimmed_series_id': 'series_id'})

# isolate years 2019 and 2020
KS_df_clean = KS_df.loc[KS_df['year'] >= 2019]

# isolate data for unemployment percentages and totals
KS_df_percents = KS_df_clean[KS_df_clean['series_id'] == 'LASST200000000000003']
KS_df_totals = KS_df_clean[KS_df_clean['series_id'] == 'LASST200000000000004']

# merge percentage and totals dataframes
KS_df_clean_merged = KS_df_percents.merge(KS_df_totals, how='outer', on = ['period', 'year'])

# rename columns for readability
KS_df_clean_renamed = KS_df_clean_merged.rename(columns = {'series_id_x': 'series_id_percent',
                                    'period_x': 'period_percent',
                                    'value_x': 'value_percent',
                                    'series_id_y': 'series_id_total',
                                    'period_y': 'period_total',
                                    'value_y': 'value_total'
                                    })
# move columns for readability
KS_df_columnMoved = KS_df_clean_renamed[['year', 'period', 'series_id_percent', 'value_percent', 'series_id_total', 'value_total']]

# rename columns for readability
KS_df_full = KS_df_columnMoved.rename(columns = {'year': 'Year',
                                                           'period': 'Month',
                                                           'series_id_percent': 'Unemployment Percentage ID',
                                                           'value_percent': 'Unemployment Percentage',
                                                           'series_id_total': 'Unemployment Total ID',
                                                           'value_total': 'Total Unemployment'
                                                          })
# drop the ID columns as they are no longer needed in the data
KS_df_short = KS_df_full.drop(columns = ['Unemployment Percentage ID', 'Unemployment Total ID'])

# drop unnecessary columns
MO_df = MO_df.drop(columns = ['footnote_codes', 'series_id'])

# rename trimmed series id
MO_df = MO_df.rename(columns = {'trimmed_series_id': 'series_id'})

# isolate years 2019 and 2020
MO_df_clean = MO_df.loc[MO_df['year'] >= 2019]

# isolate the unemployment percentage data and total unemployment numbers
MO_df_percents = MO_df_clean[MO_df_clean['series_id'] == 'LASST290000000000003']
MO_df_totals = MO_df_clean[MO_df_clean['series_id'] == 'LASST290000000000004']

# merge total and percent dataframes
MO_df_clean_merged = MO_df_percents.merge(MO_df_totals, how='outer', on = ['period', 'year'])

# rename columns to better reflect data
MO_df_clean_renamed = MO_df_clean_merged.rename(columns = {'series_id_x': 'series_id_percent',
                                    'period_x': 'period_percent',
                                    'value_x': 'value_percent',
                                    'series_id_y': 'series_id_total',
                                    'period_y': 'period_total',
                                    'value_y': 'value_total'
                                    })
# move columns for readability
MO_df_columnMoved = MO_df_clean_renamed[['year', 'period', 'series_id_percent', 'value_percent', 'series_id_total', 'value_total']]

# rename columns for readability
MO_df_full = MO_df_columnMoved.rename(columns = {'year': 'Year',
                                                           'period': 'Month',
                                                           'series_id_percent': 'Unemployment Percentage ID',
                                                           'value_percent': 'Unemployment Percentage',
                                                           'series_id_total': 'Unemployment Total ID',
                                                           'value_total': 'Total Unemployment'
                                                          })
# drop ID columns to clean data
MO_df_short = MO_df_full.drop(columns = ['Unemployment Percentage ID', 'Unemployment Total ID'])

# merge KS and MO dataframes
KS_MO_Merged = KS_df_short.merge(MO_df_short, how='outer', on = ['Year', 'Month'])

# rename columns to reflect data merge
KS_MO_Unemployment_Data = KS_MO_Merged.rename(columns = {'Unemployment Percentage_x': 'Kansas Unemployment (%)',
                                                           'Total Unemployment_x': 'Kansas Total Unemployment',
                                                           'Unemployment Percentage_y': 'Missouri Unemployment (%)',
                                                           'Total Unemployment_y': 'Missouri Total Unemployment'  
                                                            })
# replace month codes with months for readability
KS_MO_Final_df = KS_MO_Unemployment_Data.replace(['M01',
                                  'M02',
                                  'M03',
                                  'M04',
                                  'M05',
                                  'M06',
                                  'M07',
                                  'M08',
                                  'M09',
                                  'M10',
                                  'M11',
                                  'M12'],
                                 
                                 ['Jan',
                                  'Feb',
                                  'Mar',
                                  'Apr',
                                  'May',
                                  'Jun',
                                  'Jul',
                                  'Aug',
                                  'Sep',
                                  'Oct',
                                  'Nov',
                                  'Dec',
                                 ])
# present final dataframe
KS_MO_Final_df

# export as csv
# KS_MO_Final_df.to_csv('Kansas_Missouri_Unemployment_Data_Cleaned.csv')

['series_id                     ',
 'year',
 'period',
 '       value',
 'footnote_codes']